**Imports**

In [1]:
import numpy as np
import pandas as pd
import os 

import tensorflow as tf

from sbb import Sbb

**Global variables**

In [2]:
model_folder = '../model'

checkpoint_path = model_folder + "/sbb.ckpt"

output_size = 1
input_size = (4, )

In [3]:
if not os.path.exists(model_folder):
    os.mkdir(model_folder)

In [4]:
physical_devices = tf.config.list_physical_devices("GPU")

if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

**Retrive data from system**

In [5]:
data = pd.read_csv('../data/output.csv').dropna()
train_count = round(len(data) * 0.8)

train_x, train_y = data.iloc[:train_count, :4].to_numpy(), data.iloc[:train_count, 4:5].to_numpy()
val_x, val_y = data.iloc[train_count:, :4].to_numpy(), data.iloc[train_count:, 4:5].to_numpy()

In [6]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
test_dataset = tf.data.Dataset.from_tensor_slices((val_x, val_y))

In [7]:
train_dataset

<TensorSliceDataset shapes: ((4,), (1,)), types: (tf.float64, tf.float64)>

In [8]:
sbb = Sbb(checkpoint_path, output_size, input_size)

In [9]:
sbb.train(train_dataset, test_dataset, epochs=2000)

(4,)
Model: "DENSE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
DENSE-1-DENSE-1 (Dense)      (None, 256)               1280      
_________________________________________________________________
DENSE-1-NORM-1 (BatchNormali (None, 256)               1024      
_________________________________________________________________
DENSE-1-DROP-1 (Dropout)     (None, 256)               0         
_________________________________________________________________
DENSE-1-DENSE-2 (Dense)      (None, 256)               65792     
_________________________________________________________________
DENSE-1-NORM-2 (BatchNormali (None, 256)               1024      
_________________________________________________________________
DENSE-1-DROP-2 (Dropout)     (None, 256)               0

ValueError: in user code:

    /Users/joel/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /Users/joel/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/joel/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/joel/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/joel/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/joel/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py:835 ru

In [ ]:
model.evaluate(test_dataset)